In [12]:
import carla
import random
import time
import os
import signal
import numpy as np
import tempfile
import pickle

from norospy import ROSFoxgloveClient

from mmdet3d.apis import MonoDet3DInferencer
from typing import Callable, Dict, Set, Tuple, Any

In [2]:
client = carla.Client("localhost", 2000)

In [3]:
world = client.load_world("Town03")

In [4]:
bp_lib = world.get_blueprint_library()

In [5]:
vehicle_bp = bp_lib.filter("vehicle")
spawn_pts = world.get_map().get_spawn_points()

In [6]:
ego_vehicle = world.spawn_actor(random.choice(vehicle_bp), random.choice(spawn_pts))

In [7]:
# Spawn 50 vehicles randomly distributed throughout the map 
# for each spawn point, we choose a random vehicle from the blueprint library
for i in range(0,50):
    world.try_spawn_actor(random.choice(vehicle_bp), random.choice(spawn_pts))

In [9]:
for vehicle in world.get_actors().filter('*vehicle*'):
    vehicle.set_autopilot(True)

In [8]:
spectator = world.get_spectator()

In [9]:
transform = ego_vehicle.get_transform()
spectator.set_transform(carla.Transform(transform.location + carla.Location(z=2), carla.Rotation(yaw=90)))

In [11]:
cam_bp = None
# TODO: set proper transformation to the cam
cam_init_transform = carla.Transform(carla.Location(z=3))
cam_bp = world.get_blueprint_library().find("sensor.camera.rgb")
cam = world.spawn_actor(cam_bp, cam_init_transform, attach_to=ego_vehicle)
cam.attributes

time.sleep(3)
cam.destroy

<bound method destroy of <carla.libcarla.ServerSideSensor object at 0x76676858af90>>

In [3]:
# initialize the model
config_path = "/home/ws/uqmfs/mmdetection3d/configs/fcos3d/fcos3d_r101-caffe-dcn_fpn_head-gn_8xb2-1x_nus-mono3d_finetune.py"
checkpoint_path = "/home/ws/uqmfs/mmdetection3d/weights/fcos3d_r101_caffe_fpn_gn-head_dcn_2x8_1x_nus-mono3d_finetune_20210717_095645-8d806dc2.pth"

inferencer = MonoDet3DInferencer(config_path, checkpoint_path)

Loads checkpoint by local backend from path: /home/ws/uqmfs/mmdetection3d/weights/fcos3d_r101_caffe_fpn_gn-head_dcn_2x8_1x_nus-mono3d_finetune_20210717_095645-8d806dc2.pth
05/22 17:12:54 - mmengine - WARNING - Failed to search registry with scope "mmdet3d" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet3d" is a correct scope, or whether the registry is initialized.
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/ws/uqmfs/miniconda3/envs/openmmlab/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [ ]:
def modify_anotation(image_path, camera_matrix):
    infos = {}
    infos={
        'data_list': [
            {
                'images': {
                    'CAM2': {
                        'img_path': image_path,
                        'cam2img': camera_matrix,
                        'lidar2img': np.array([]),
                        'lidar2cam': np.array([]),
                    }
                }
            }
        ]
    }
    infos_file = tempfile.NamedTemporaryFile(delete=False, suffix='.pkl')
    pickle.dump(infos, infos_file)
    infos_file.close()
    return infos_file

In [13]:
def online_inference(msg: Any, ts: int):
    # reshape the img to (w, h, 4)
    img = np.array(msg.raw_data).reshape(msg.width, msg.height, -1)
    # projection/camera matrix from ros topic
    P = np.array([400.00000000000006, 0, 400, 0, 0, 400.00000000000006, 300, 0, 0, 0, 1, 0], dtype=np.float64).reshape(3,4)
    C = np.array([400.00000000000006, 0, 400, 0, 400.00000000000006, 300, 0, 0, 1], dtype=np.float64).reshape(3,3)
    
    # TODO: construct the pkl file based on the camera calibration
    

    input = dict(img=img, infos=)
    results = inferencer(input, return_datasamples=True, show=True)

In [ ]:
try:
    client = ROSFoxgloveClient('ws://localhost:8765')
    client.run_background()
    client.subscribe('/carla/autopilot/front/image', 'sensor_msgs/Image', online_inference)
    signal.pause()
finally:
    client.close()